In [113]:
# Imports

import car
from auto.capabilities import acquire, release, list_caps
from car.motors import set_throttle, set_steering
from auto.camera import global_camera, close_global_camera
from auto import console
from typing import Tuple
import time

import numpy as np
import cv2

list_caps()
cv2.setNumThreads(0)

In [114]:
# Hardware Constants
STRAIGHT_STEER_BIAS=-6.0
STRAFE_ANGLE=5.0
CORNER_STEER_MULTIPLIER=1.298
CORNER_STEER_BIAS=0.457

# Hues, takes these and divide by 2
GREEN_HUE=175
BLUE_HUE=215
RED_HUE=345
WHITE_HUE=215
PURPLE_HUE=280
CAR_HUE=-1

# Color Diffs
TRACK_DIFF=5
WHITE_DIFF=25
CAR_DIFF=0

# Color Thresholds
RED_TURN_THRESHOLD=0.7
NONE_THRESHOLD=0.7 # Technically could just be the same as the RED_TURN_THRESHOLD
PURPLE_TURN_THRESHOLD=0.6
WHITE_STRAIGHT_THRESHOLD=0.4
GREEN_LEFT_THRESHOLD=0.1
BLUE_RIGHT_THRESHOLD=0.1
CAR_THRESHOLD=10000000 # Furthest out
OVERTAKE_THRESHOLD=10000000 # Medium
TOO_CLOSE_THRESHOLD=10000000 # Closest and dangerous

# Racing
NUM_LAPS=5
THROTTLE_OFFSET=-20.0
MIN_THROTTLE=15
MAX_THROTTLE=22

# Physics
STATIC_FRICTION=0.7
GRAVITY=32.2
WHEEL_BASE=0.5

In [115]:
# Functions

def track_bounds_crop(img):
    """
    Camera captures 240 pixels of height and 320 of width
    Shape is (240, 320, 3)
    """
    return img[-70:, 60:-60]


def track_zones_crop(img):
    """
    Cut out the top half
    """
    return img[-100:, :]


def car_location_crop(img):
    """
    Cut top and bottom thirds
    """
    return img[80:-80, :]


def percent_color(img, target_hue, diff):
    """
    Returns the percentage of the image that is the color using HSV
    """
    hsv_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h, _, _ = cv2.split(hsv_image)

    # Circular hue distance
    dist = np.abs(h - target_hue)
    dist = np.minimum(dist, 180 - dist)   # wrap-around fix

    # Pixels within threshold
    mask = (dist < diff).astype(np.uint8) * 255

    color_percent = cv2.countNonZero(mask)/(img.size/3)
    return color_percent


def throttle_to_feet_per_second(throttle):
    return 0.33514 * throttle - 3.39442


def feet_per_second_to_throttle(feet_per_second):
    return (feet_per_second + 3.39442) / 0.33541


def adjust_radius(inner_radius: float, r1: float, r2: float, angle: float) -> float:
    """
    Adjust the radius of the path taken to end up r2 away from the inner radius when starting r1 away from the inner radius
    r1 is the radius adjustment entering the turn (so average(inner_radius, outer_radius) - inner_radius would be r1 when the car starts in the middle)
    r2 is the radius adjustment exiting the turn
    angle is in radians
    """
    return np.sqrt((inner_radius + r1) * (inner_radius + r2) + ((r1 - r2) / (2 * np.sin(angle / 2)))**2)


def too_close(frame):
    """
    If a car ahead is too close, then it is the duty of the car behind to slow down
    """
    middle_width = 140

    l_bound = 160 - (middle_width / 2)
    r_bound = 160 + (middle_width / 2)

    if percent_color(frame[:, l_bound:r_bound], CAR_HUE, CAR_DIFF) > TOO_CLOSE_THRESHOLD:
        return True
    else:
        return False


def can_overtake(frame) -> bool:
    """
    Check if there is a car that can be overtaken
    """
    middle_width = 140

    l_bound = 160 - (middle_width / 2)
    r_bound = 160 + (middle_width / 2)

    if percent_color(frame[:, l_bound:r_bound], CAR_HUE, CAR_DIFF) > OVERTAKE_THRESHOLD:
        return True
    else:
        return False
    

def identify_opposing_cars(frame) -> Tuple[bool, bool, bool]:
    """
    Returns three values:
    1. Is there a car to the left of the path you want to take
    2. Is there a car directly on the path you want to take
    3. Is there a car to the right of the path you want to take
    """
    middle_width = 140

    l_bound = 160 - (middle_width / 2)
    r_bound = 160 + (middle_width / 2)

    left_zone   = frame[:, :l_bound]
    middle_zone = frame[: l_bound:r_bound]
    right_zone  = frame[: r_bound:]

    left_percent   = percent_color(left_zone, CAR_HUE, CAR_DIFF)
    middle_percent = percent_color(middle_zone, CAR_HUE, CAR_DIFF)
    right_percent  = percent_color(right_zone, CAR_HUE, CAR_DIFF)

    return (left_percent > CAR_THRESHOLD, middle_percent > CAR_THRESHOLD, right_percent > CAR_THRESHOLD)

In [116]:
# console.clear()
# while True:
#     frame = car.capture()
#     img = track_zones_crop(frame)
#     blue_percent = percent_color(img, BLUE_HUE, TRACK_DIFF)
#     green_percent = percent_color(img, GREEN_HUE, TRACK_DIFF)
#     red_percent = percent_color(img, RED_HUE, TRACK_DIFF)
#     white_percent = percent_color(img, WHITE_HUE, TRACK_DIFF)
#     purple_percent = percent_color(img, PURPLE_HUE, TRACK_DIFF)
#     car_percent = percent_color(img, CAR_HUE, CAR_DIFF)
#     console.clear()
#     console.print(f"Blue: {blue_percent}\nGreen: {green_percent}\nRed: {red_percent}\nWhite: {white_percent}\nPurple: {purple_percent}\nCar: {car_percent}")

In [ ]:
# Racing variables

# It is more consistent to follow the outer radius, but the inner radius can be useful in inside lines etc
# inner_radii = [2.352288712045608, 1.206350190233798, 1.6627305093830878, 1.0541789388712448, 0.47572180000000003, 0.5867929778501835, 0.8351655163157062, 1.027879788271899, 1.5126722175388703, 1.634663057576926, 2.3622048]
# outer_radii = [5.63312871, 4.48719019, 4.94357051, 3.51480894, 2.4442258, 3.86763298, 4.11600552, 3.48850979, 4.46542822, 4.75146106, 5.6430448]

inner_radii = [3.352288712045608, 1.206350190233798, 1.6627305093830878, 1.0541789388712448, 0.47572180000000003, 0.5867929778501835, 0.8351655163157062, 1.027879788271899, 1.5126722175388703, 1.634663057576926, 2.3622048]
outer_radii = [6.63312871, 4.48719019, 4.94357051, 3.51480894, 2.4442258, 3.86763298, 4.11600552, 3.48850979, 4.46542822, 4.75146106, 5.6430448]

angles = [196, -90, 74, -66, 180, -114, 90, -100, -57, 67, 180]
color_order = ["RED", "NONE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "PURPLE", "WHITE", "RED", "NONE", "WHITE"] # Fill with the order in which the colors come for the zones, a NONE comes after a red zone without a purple immediately after

num_turns = len(angles)

In [118]:
# Custom logic
class LoopInputs:
    def __init__(self, too_close: bool, can_overtake: bool, cars: Tuple[bool, bool, bool], state: str, turn_index: int, lap_number: int, angle_accum: float, track_pos: str):
        self.too_close = too_close
        self.can_overtake = can_overtake
        self.cars = cars
        self.state = state
        self.turn_index = turn_index
        self.lap_number = lap_number
        self.angle_accum = angle_accum
        self.track_pos = track_pos


class Car:
    def __init__(self):
        self.instruction = {} # Any arbitrary instructions the user wants can be thrown in here, like {"action": "strafe", "start": 2 seconds ago, "duration": 5 seconds, "angle": -5}

        self.state = "STRAIGHT"
        self.next_color = "RED" # Either "RED", "WHITE", "PURPLE", or "NONE"
        self.throttle = MIN_THROTTLE # This is always MIN_THROTTLE at the start for testing purposes
        self.angle_accum = 0
        self.lap_number = 0
        self.color_index = 0
        self.turn_index = num_turns - 1

        # Hardware
        self.gyroscope = acquire('Gyroscope_accum')
        self.gyroscope.read()

        camera = global_camera(False)
        self.stream = camera.stream()
        next(self.stream)
    

    def fetch_inputs(self):
        """
        Returns all of the inputs needed to control the car
        """
        if self.stream:
            frame = next(self.stream)
        else:
            return None
        
        # Check for instructions to change state
        bottom_portion_img = track_zones_crop(frame)
        switch_color = False
        if self.next_color == "RED":
            # Look for red braking zones
            red_percent = percent_color(bottom_portion_img, RED_HUE, TRACK_DIFF)
            if red_percent > RED_TURN_THRESHOLD:
                next_color_index = (self.color_index + 1) % len(color_order)
                self.state = "BRAKE" if color_order[next_color_index] == "NONE" else "TURN"
                self.turn_index = (self.turn_index + 1) % num_turns
                switch_color = True
        elif self.next_color == "NONE":
            # Wait for there to be no more red
            red_percent = percent_color(bottom_portion_img, RED_HUE, TRACK_DIFF)
            if red_percent < NONE_THRESHOLD:
                self.state = "TURN"
                switch_color = True
        elif self.next_color == "WHITE":
            # Look for white braking zones
            white_percent = percent_color(bottom_portion_img, WHITE_HUE, TRACK_DIFF)
            if white_percent > WHITE_STRAIGHT_THRESHOLD:
                self.state = "STRAIGHT"
                self.angle_accum += angles[self.turn_index]
                switch_color = True
                if self.turn_index == num_turns - 1:
                    self.lap_number += 1
        else:
            # Look for purple braking zones, these will always come after red and be turns
            purple_percent = percent_color(bottom_portion_img, PURPLE_HUE, TRACK_DIFF)
            if purple_percent > PURPLE_TURN_THRESHOLD:
                self.state = "TURN"
                switch_color = True
                self.angle_accum += angles[self.turn_index]
                self.turn_index = (self.turn_index + 1) % num_turns

        if switch_color:
            self.color_index = (self.color_index + 1) % len(color_order)
            self.next_color = color_order[self.color_index]

        track_bounds_img = track_bounds_crop(frame)
        if percent_color(track_bounds_img, GREEN_HUE, TRACK_DIFF) > GREEN_LEFT_THRESHOLD:
            track_pos = "LEFT"
        elif percent_color(track_bounds_img, BLUE_HUE, TRACK_DIFF) > BLUE_RIGHT_THRESHOLD:
            track_pos = "RIGHT"
        else:
            track_pos = "MIDDLE"

        car_cropped_img = car_location_crop(frame)
        
        return LoopInputs(too_close(car_cropped_img), can_overtake(car_cropped_img), identify_opposing_cars(car_cropped_img), self.state, self.turn_index, self.lap_number, self.angle_accum, track_pos)


    def loop(self, inputs: LoopInputs) -> Tuple[float, float]:
        """
        User determines the throttle and steering
        """
        true_max = MAX_THROTTLE
        steer_direction = 1

        if inputs.state == "STRAIGHT":
            steer_bias = STRAIGHT_STEER_BIAS
            angle_offset = 0

            # Check if a car can be overtaken
            if inputs.can_overtake and not self.instruction.get("action") == "overtake":
                true_car_position = None
                if inputs.cars[1]:
                    true_car_position = inputs.track_pos
                elif inputs.cars[0]:
                    # This isn't actually true, but in terms of how the racing will go, this is a good approximation
                    true_car_position = "LEFT"
                else:
                    # This isn't actually true, but in terms of how the racing will go, this is a good approximation
                    true_car_position = "RIGHT"

                if true_car_position == "LEFT":
                    # If the opposing car is ahead on the left, move to the right side
                    self.instruction = {"action": "overtake", "start": time.time(), "direction": "right"}
                else:
                    # Otherwise go on the inside
                    self.instruction = {"action": "overtake", "start": time.time(), "direction": "left"}
            elif not inputs.can_overtake and (inputs.bound == "LEFT" or inputs.bound == "RIGHT") and not self.instruction.get("action") == "strafe":
                # Move to the middle of the track
                width = outer_radii[self.turn_index] - inner_radii[self.turn_index]
                velocity = throttle_to_feet_per_second(self.throttle)
                self.instruction = {"action": "strafe", "start": time.time(), "duration": width / (2 * velocity * np.sin(np.pi / 180 * STRAFE_ANGLE)), "angle": STRAFE_ANGLE * (1 if inputs.bound == "LEFT" else -1)}

            if self.instruction.get("action") == "overtake":
                true_max += 5 # Can go faster while overtaking on straights
                if time.time() - self.instruction.get("start") > 10:
                    # If going for more than 10 seconds, stop the attempt and add a late braking move (likely something has gone wrong, but who really cares)
                    self.instruction = {"action": "late_brake", "start": None, "duration": 0.5}
                elif inputs.bound == "LEFT" and self.instruction.get("direction") == "left" or inputs.bound == "RIGHT" and self.instruction.get("direction") == "right":
                    # If the car has ended up on the edge, make a small adjustment away from the edge for consistency then add a late braking move to have the overtake stick
                    self.instruction = {"action": "adjust", "start": time.time(), "duration": 1.0, "angle": STRAFE_ANGLE * (1 if inputs.bound == "RIGHT" else -1), "next_instruction": "late_brake"}
                else:
                    angle_offset = STRAFE_ANGLE
                    if self.instruction.get("direction") == "left":
                        angle_offset *= -1
            elif self.instruction.get("action") == "strafe":
                if time.time() - self.instruction.get("start") > self.instruction.get("duration"):
                    self.instruction = {}
                elif inputs.bound == "LEFT" or inputs.bound == "RIGHT":
                    # If we hit the bounds we have overshot, so make a small adjustment away from the edge
                    self.instruction = {"action": "adjust", "start": time.time(), "duration": 1.0, "angle": STRAFE_ANGLE * (1 if inputs.bound == "RIGHT" else -1), "next_instruction": None}
                else:
                    angle_offset = self.instruction.get("angle")
            elif self.instruction.get("action") == "adjust":
                if time.time() - self.instruction.get("start") > self.instruction.get("duration"):
                    if self.instruction.get("next_instruction") == "late_brake":
                        self.instruction = {"action": "late_brake", "start": None, "duration": 0.5}
                    else:
                        self.instruction = {}
                else:
                    angle_offset = self.instruction.get("angle")
            
            z = self.gyroscope.read()[2]
            steer = -1 * (z - (inputs.angle_accum + angle_offset)) # From sample code on the AutoAuto website, they double the offset

            self.throttle = true_max # Try to go flat out on straights

        elif inputs.state == "BRAKE":
            # Cancel any overtake instructions
            if self.instruction.get("action") != "late_brake":
                self.instruction = {}

            steer_bias = STRAIGHT_STEER_BIAS
            
            z = self.gyroscope.read()[2]
            steer = -1 * (z - (inputs.angle_accum)) # From sample code on the AutoAuto website, they double the offset

            if self.instruction.get("action") == "late_brake":
                if self.instruction.get("start") is not None:
                    if time.time() - self.instruction.get("start") > self.instruction.get("duration"):
                        self.throttle = 0 # For now let it get clipped, but as speed builds up this throttle can change
                        self.instruction = {}
                else:
                    self.instruction["start"] = time.time()
        else:
            """
            Due to error and good racing doesn't follow the perfect line, this can be deviated from, but anything widely
            off of this angle will be very very inefficient and maybe even dangerous
            """
            steer_bias = 0
            throttle_bias = 0

            inner_radius = inner_radii[self.turn_index]
            outer_radius = outer_radii[self.turn_index]
            r1 = (inner_radius + outer_radius) / 2 - inner_radius
            if inputs.bound == "LEFT":
                r1 = 0
            elif inputs.bound == "RIGHT":
                r1 = outer_radius - inner_radius
                
            r2 = (inner_radius + outer_radius) / 2 - inner_radius 
            turn_radius = adjust_radius(inner_radius, r1, r2, angles[self.turn_index])
            
            steer_direction = 1 if angles[self.turn_index] > 0 else -1

            if steer_direction == 1 and inputs.bound == "LEFT" or steer_direction == -1 and inputs.track_pos == "RIGHT":
                # If we are on the outer radius and reach the outer line, tighten the line
                turn_radius *= 0.9
                throttle_bias = THROTTLE_OFFSET
            elif steer_direction == -1 and inputs.bound == "LEFT" or steer_direction == 1 and inputs.track_pos == "RIGHT":
                # If we are on the inner radius and reach the inner line, reverse the direction
                turn_radius *= 1.1
                steer_direction = -steer_direction
                throttle_bias = THROTTLE_OFFSET

            # Convert to deg
            steer = np.arctan(WHEEL_BASE / turn_radius) * 180 / np.pi
            steer = CORNER_STEER_MULTIPLIER * steer + CORNER_STEER_BIAS # Account for the steering bias, different for each car
            self.throttle = feet_per_second_to_throttle(np.sqrt(turn_radius * STATIC_FRICTION * GRAVITY))
            self.throttle += throttle_bias
            
        if inputs.too_close:
            true_max = 30
        self.throttle = min(max(MIN_THROTTLE, self.throttle), true_max)
        
        return steer * steer_direction + steer_bias, self.throttle

In [ ]:
console.clear()

car = Car()

throttle = MIN_THROTTLE

# This tests the min throttle, and this is necessary because the cars can be stupid and not run sometimes
for i in range(4):
    set_throttle(MIN_THROTTLE)
    z = car.gyroscope.read()[2]
    set_steering(-z)

set_throttle(0)
set_steering(0)

for i in range(10):
    print(10 - i)
    time.sleep(1)

print("GO!!!")

angle_accum = 0
lap = 0
while lap <= NUM_LAPS:
    d = car.fetch_inputs() # Obtain the track state, bounds, cars ahead, lap number, and turn index
    if d is None:
        pass
    else:
        steer, throttle = car.loop(d)
        # with open("log.txt", "a+") as f:
        #     f.write(str(steer) + ' ' + d.state + ' ' + d.bound + ' ' + str(d.turn_index) + "\n")
        if lap == NUM_LAPS:
            set_throttle(int(throttle * 0.4))  # convert to native int
        else:
            set_throttle(int(throttle))  # convert to native int
        set_steering(steer)

release(car.gyroscope)
close_global_camera(False)

KeyboardInterrupt: 